### Análisis de Resultados

Finalizado el punto anterior, seleccionar uno de los modelos y realizar el siguiente análisis (se esperan respuestas cortas y concisas)

MODELO ELEGIDO: **Random Forest** 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier

from sklearn import tree
import graphviz

In [2]:
data = pd.read_csv('../datasets/sneep-unificado-2002-2017-CURADO.csv')

/home/lucia/.anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (34,36,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
X = data[['delito1_id','provincia_id','genero_id','nacionalidad_id',
         'es_reincidente_id','anio_condenado','edad_al_ser_condenado',
          'nivel_instruccion_id','estado_civil_id']].astype(int)
y = data['duracion_condena_rango'].astype(int)

xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.10, random_state = 0)

In [4]:
# Estos son los mejores parámetros encontrados con GridSearch en la notebook anterior
best_params = {'n_estimators':450, 
               'criterion':'entropy', 
               'max_depth':20, 
               'max_features':'auto', 
               'random_state':0}

model = RandomForestClassifier(n_estimators=best_params['n_estimators'], 
                               max_depth=best_params['max_depth'],
                               random_state=best_params['random_state'],
                               max_features=best_params['max_features'],
                               criterion=best_params['criterion']
                              )
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

1- Comparar los resultados sobre el set de entrenamiento y sobre el set de test. ¿Qué podemos decir respecto a sesgo y varianza en estos resultados?

Esta pregunta está relacionada con lo que se conoce como **the bias-variance tradeoff** (https://towardsdatascience.com/random-forests-and-the-bias-variance-tradeoff-3b77fee339b4). 

Se dice que un modelo está sesgado si sistemáticamente subestima o sobrestima la variable objetivo. Esto puede deberse a las suposiciones estadísticas hechas por el modelo elegido o a algún sesgo presente en los datos de entrenamiento.

La varianza captura la capacidad de generalizar del modelo, ya que es una medida de cuánto cambiaría nuestra predicción si la entrenamos en diferentes datos. Una varianza alta indicaría que es posible que el modelo esté overfitteando. 

Por lo general, modelos con un sesgo más bajo en la estimación de los parámetros, tienen una varianza más alta y viceversa. Lo que nos gustaría, idealmente, es un sesgo bajo y una varianza baja.

A continuación definimos una función que calcula el error cuadrático medio, la varianza y el sesgo.

In [5]:
def bias_variance(y_true, y_pred, e=0.05):
    """
    Calcula el error cuadrático medio, la varianza y el sesgo
    """ 
    mse = np.mean((y_pred - y_true)**2)
    var = np.var(y_pred)
    bias = mse - var - e
    
    print("Mean square error: %.2f" % mse)
    print("Bias: {bias}".format(bias=bias))
    print("Variance: {var}".format(var=var))

In [6]:
print("-------------------------------------------------------------------------------------------------")
print("SET DE TEST:")
print("-------------------------------------------------------------------------------------------------")

bias_variance(yTest, yPred)
print("-------------------------------------------------------------------------------------------------")
print(classification_report(yTest, yPred))
print('accuracy:', accuracy_score(yTest, yPred))

-------------------------------------------------------------------------------------------------
SET DE TEST:
-------------------------------------------------------------------------------------------------
Mean square error: 2.11
Bias: -0.9525411629798166
Variance: 3.0158337749635282
-------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

           1       0.44      0.13      0.20       288
           2       0.56      0.77      0.65      2012
           3       0.51      0.47      0.49      1658
           4       0.57      0.51      0.54      1143
           5       0.61      0.41      0.49       561
           6       0.69      0.43      0.53       285
           7       0.67      0.67      0.67       929

    accuracy                           0.57      6876
   macro avg       0.58      0.48      0.51      6876
weighted avg       0.57      0.57      0.56      6876

accuracy: 0.57068

In [ ]:
print("-------------------------------------------------------------------------------------------------")
print("SET DE ENTRENAMIENTO:")
print("-------------------------------------------------------------------------------------------------")

yPred2 = model.predict(xTrain)

bias_variance(yTrain, yPred2)
print("-------------------------------------------------------------------------------------------------")
print(classification_report(yTrain, yPred2))
print('accuracy:', accuracy_score(yTrain, yPred2))

-------------------------------------------------------------------------------------------------
SET DE ENTRENAMIENTO:
-------------------------------------------------------------------------------------------------


Obtuvimos una accuracy del 57% en el set de test, mientras que en el conjunto de entrenamiento la accuracy fué del 85%. También precision y recall dieron más alto en el conjunto de entrenamiento que en el de test (86% y 85% contra 57% y 57%). 

Por otro lado, el error cuadrático medio es mucho mayor en el set de prueba (2.11 contra 0.69), mientras que la varianza es casi la misma en ambos sets (3.02 y 3.06). 

2- Imprimir la matriz de confusión de los resultados sobre el set de test. Desarrollar un análisis corto de lo que ven.

Definimos una función para graficar la **matriz de confusión**, esto nos ayudará a visualizar mejor los resultados.

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
class_names = ['1', '2', '3', '4', '5', '6', '7']

# Plot non-normalized confusion matrix
plot_confusion_matrix(yTest, yPred, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plot_confusion_matrix(yTest, yPred, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.tight_layout()  # Optional ... often improves the layout 

Se observa que el modelo entrenado clasifica bastante bien a las clases '2', '4' y '7' y bastante mal a la clase '1' (casi siempre sobrestima hacia la clase '2'). El resto de las clases las clasifica mediocremente (ni tan bien, ni tan mal).

3- Imprimir en una tabla (pueden crear un dataframe con los datos) la importancia relativa para el modelo de los distintos atributos (feature_importances_). Nuevamente, escribir un comentario respecto al resultado.

https://towardsdatascience.com/explaining-feature-importance-by-example-of-a-random-forest-d9166011959e

https://towardsdatascience.com/feature-selection-techniques-in-machine-learning-with-python-f24e7da3f36e

### Mejora del Modelo

1- Luego de realizado el análisis, es momento de intentar mejorar nuestro modelo. En ésta oportunidad, ustedes deben elegir al menos dos cambios a realizar para intentar mejorar los resultados. Los cambios pueden ser (pero no están limitados a): 

- Agregar o quitar variables para entrenamiento (incluyendo otros delitos)
- Cambiar el encoding de las variables
- Recuperar registros que se filtraron al principio para tener más ejemplos
- Filtrar mayor cantidad de registros
- Generar una nueva variable (como se hizo con edad_al_ser_condenado)


### Conclusiones

Escribir una conclusión breve respecto al trabajo. Particularmente sus ideas respecto a la pregunta inicial:
¿Es posible predecir la duración de la condena que recibirá una persona en base a datos como los que se encuentran presentes en este dataset?

Mencionar: 
- La calidad de los resultados obtenidos.
- Causas posibles de problemas que hayan detectado.
- Posibles soluciones